In [ ]:
import torch
import torch.nn as nn
import torchvision
import numpy as np
import copy
import random
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import Sampler
from torch.utils.data import BatchSampler, RandomSampler, WeightedRandomSampler
from torch import linalg as la


class LogisticRegression(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegression, self).__init__()
        self.l1 = nn.Linear(input_dim, output_dim)
        self.sigmoid = nn.Sigmoid()
    def forward(self, x):
        x = self.l1(x)
        x = self.sigmoid(x)
        return x 


#Imports ijcnn dataset in libsvm format
class LIBSVMdataset(torch.utils.data.Dataset):
    def __init__(self, file_path, n_features, n_sizes):
        self.file_path = file_path
        self.n_features = n_features
        self.n_sizes = n_sizes
        with open(self.file_path, "r") as fp:
            self.dmatrix = fp.readlines()
        
    def label_transformer(self, l):
        if l != 1:
            l = 0
        return l
    
    def process_line(self, line):
        line = line.split(' ')
        label, values = int(self.label_transformer(float(line[0]))), line[1:]
        value = torch.zeros((self.n_features))
        for item in values:
            idx, val = item.split(':')
            value[int(idx)-1] = float(val)
        return [label, value]
    
    def __len__(self):
        return int(self.n_sizes)
    
    def __getitem__(self, number):
        target, features = self.process_line(self.dmatrix[number].strip("\n"))
        return target, features


#Resets gradients of all model parameters
def zero_grad(params):
    for p in params:
        if p.grad is not None:
            p.grad.detach()
            p.grad.zero_()

            
#Records the training losses
def total_loss(model, loss_fns, dataloader, n):
    total_loss = 0
    for y, x in dataloader:
        out = model(x)
        loss = loss_fns(out, y.float().view(x.shape[0], -1))
        total_loss = total_loss + loss.item()
    return total_loss * (1/n)


#Records the testing losses
def test_loss(model, loss_fns, test_dataloader, n):
    test_loss = 0
    for y, x in test_dataloader:
        out = model(x)
        loss = loss_fns(out, y.float().view(x.shape[0], -1))
        test_loss = test_loss + loss.item()
    return test_loss * (1/n)


#Records the l2 norm of gradients
def total_grad(model, loss_fns, dataloader, n):
    total_grad = 0
    zero_grad(list(model.parameters()))
    for y, x in dataloader:
        out = model(x)
        loss = loss_fns(out, y.float().view(x.shape[0], -1))
        loss.backward()
    for p in list(model.parameters()):
        total_grad = total_grad + torch.sum(torch.square(torch.mul(torch.clone(p.grad.data).detach(), (1/n))))
    zero_grad(list(model.parameters()))
    return torch.sqrt(torch.clone(total_grad).detach())


#Records the full gradients (used in SVRG and GGD-SVRG algorithms)
def full_grad(model, loss_fns, dataloader, n):
    full_grad = []
    zero_grad(list(model.parameters()))
    for y, x in dataloader:
        out = model(x)
        loss = loss_fns(out, y.float().view(x.shape[0], -1))
        loss.backward()
    for p in list(model.parameters()):
        full_grad.append(torch.mul(torch.clone(p.grad.data).detach(), (1/n)))
    zero_grad(list(model.parameters()))
    return full_grad # a list of model parameters



def get_default_device():
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')

    

def to_device(data, device):
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)



class DeviceDataLoader():
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
    def __iter__(self):
        for b in self.dl:
            yield to_device(b, self.device)
    def __len__(self):
        return len(self.dl)
    

#preparation for training 
d = 22 
n = 49990 
n1 = 91701
#Regularization parameter
weight_decay = 1/n
epoches = 30
rec = 512
#Loss function that is used to record the experiment results.
loss_func_rec = nn.BCELoss(reduction = 'sum')
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#Datasets which are used for training and testing respectively
ijcnn_train = LIBSVMdataset("./ijcnn1", d, n)
ijcnn_test = LIBSVMdataset("./ijcnn1.t", d, n1)
#Dataloaders which are used to record the training, testing losses and l2-norm of full gradients.
ijcnn_train_loader = DataLoader(ijcnn_train, batch_size = rec)
ijcnn_train_loader = DeviceDataLoader(ijcnn_train_loader, device)
ijcnn_test_loader = DataLoader(ijcnn_test, batch_size = rec)
ijcnn_test_loader = DeviceDataLoader(ijcnn_test_loader, device)

In [ ]:
#training stage: logistic regression with l^2 regularization using ggd-as
#stage one: preparation, initialization and hyperparameter setting
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
LR_net = LogisticRegression(input_dim = d, output_dim = 1)
LR_net.to(device)
copied_model = copy.deepcopy(LR_net)
copied_model.to(device)
loss_func = nn.BCELoss()
lr0 = 0.6
#Setting lr_schedule = 'constant' can obtain the experiment results for GGD
lr_schedule = 't-inverse'
b = 2
m = 16
max_batch_size = int(n/(b*m))
ijcnn_ggdas_loss_list = []
ijcnn_ggdas_gradnorm_list = []
ijcnn_ggdas_test_loss_list = []


#stage two: load training set and testing set
BS = BatchSampler(RandomSampler(ijcnn_train, replacement = False, num_samples = b*m), batch_size = b*m, drop_last = False)
LR_ijcnn_train_loader = DataLoader(ijcnn_train, batch_sampler = BS)
LR_ijcnn_train_loader = DeviceDataLoader(LR_ijcnn_train_loader, device)





for epoch in range(epoches*max_batch_size):
    LR_net.train()
    for y_target, x_data in LR_ijcnn_train_loader:
        xt = []
        yt = []
        losst = torch.empty(2)
        xt = x_data.split(m, dim = 0)
        yt = y_target.split(m, dim = 0)
        #calculate loss for first to derive the resampling probability
        for i, x in enumerate(xt):
            with torch.no_grad():
                output = LR_net(x)
                losst[i] = loss_func(output, yt[i].float().view(x.shape[0], -1)).item()
        prob = losst/torch.sum(losst)
        zero_grad(list(LR_net.parameters()))
        zero_grad(list(copied_model.parameters()))
        if lr_schedule == 't-inverse':
            lr = lr0 * (1/(1 + int(epoch/(3*max_batch_size)) ))
        else:
            lr = lr0
        #construct the grafting gradient
        output1 = LR_net(xt[0])
        output2 = copied_model(xt[1])
        loss1 = loss_func(output1, yt[0].float().view(xt[0].shape[0], -1))
        loss2 = loss_func(output2, yt[1].float().view(xt[1].shape[0], -1))
        loss1.backward()
        loss2.backward()
        for  p1, p2 in zip(list(LR_net.parameters()), list(copied_model.parameters())):
            d_p1 = p1.grad.data
            d_p2 = p2.grad.data
            if weight_decay != 0:
                d_p1.add_(p1.data, alpha = weight_decay)
                d_p2.add_(p2.data, alpha = weight_decay)
            indices = torch.zeros_like(torch.clone(d_p1).detach())
            indices = indices.bernoulli_(p = prob[0]).to(torch.bool)
            d_p1.masked_fill_(~indices, 0)
            d_p2.masked_fill_(indices, 0)
            d_p1.mul_(1/b).mul_(1/prob[0])
            d_p2.mul_(1/b).mul_(1/prob[1])
            p1.data.add_(torch.add(d_p1, d_p2), alpha = -lr)
        copied_model = copy.deepcopy(LR_net)
        copied_model.to(device)    
    if (epoch+1) % max_batch_size == 0:
        LR_net.eval()
        current_gradnorm = total_grad(LR_net, loss_func_rec, ijcnn_train_loader, n)
        ijcnn_ggdas_gradnorm_list.append(current_gradnorm)
        with torch.no_grad():
            current_loss = total_loss(LR_net, loss_func_rec, ijcnn_train_loader, n)
            ijcnn_ggdas_loss_list.append(current_loss)
            current_test_loss = test_loss(LR_net, loss_func_rec, ijcnn_test_loader, n1)
            ijcnn_ggdas_test_loss_list.append(current_test_loss)
        current_iteration =  int((epoch+1)/max_batch_size)
        print('Iteration: {}  Loss: {}  Gradnorm:{}'.format(current_iteration, current_loss, current_gradnorm))
    
            

In [ ]:
#training stage: logistic regression with l^2 regularization using ggd-adam 
#stage one: preparation, initialization and hyperparameter setting
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
LR_net = LogisticRegression(input_dim = d, output_dim = 1)
LR_net.to(device)
copied_model = copy.deepcopy(LR_net)
copied_model.to(device)
loss_func = nn.BCELoss()
lr0 = 0.005
lr_schedule = 'constant'
b = 2
m = 16
max_batch_size = int(n/(b*m))
beta_1 = torch.tensor(0.9)
beta_2 = torch.tensor(0.999)
sigma = torch.tensor(1e-8)


ijcnn_gadam_loss_list = []
ijcnn_gadam_gradnorm_list = []
ijcnn_gadam_test_loss_list = []


#stage two: load training set and testing set
BS = BatchSampler(RandomSampler(ijcnn_train, replacement = False, num_samples = b*m), batch_size = b*m, drop_last = False)
LR_ijcnn_train_loader = DataLoader(ijcnn_train, batch_sampler = BS)
LR_ijcnn_train_loader = DeviceDataLoader(LR_ijcnn_train_loader, device)


#stage three: train and test 
batch_idx = torch.tensor(0)
h_0 = [torch.zeros_like(paras) for paras in list(LR_net.parameters())]
v_0 = [torch.zeros_like(paras) for paras in list(LR_net.parameters())]
for epoch in range(epoches*max_batch_size):
    LR_net.train()
    for y_target, x_data in LR_ijcnn_train_loader:
        xt = []
        yt = []
        losst = torch.empty(2)
        xt = x_data.split(m, dim = 0)
        yt = y_target.split(m, dim = 0)
        if lr_schedule == 't-inverse':
            lr = lr0 * (1/int(1 + epoch + batch_idx/n))
        else:
            lr = lr0
        #calculate loss for first to derive the resampling probability
        for i, x in enumerate(xt):
            with torch.no_grad():
                output = LR_net(x)
                losst[i] = loss_func(output, yt[i].float().view(x.shape[0], -1)).item()
        prob = losst/torch.sum(losst)
        zero_grad(list(LR_net.parameters()))
        zero_grad(list(copied_model.parameters()))
        #construct the adam-based grafting gradient 
        output1 = LR_net(xt[0])
        loss1 = loss_func(output1, yt[0].float().view(xt[0].shape[0], -1))
        loss1.backward()
        output2 = copied_model(xt[1])
        loss2 = loss_func(output2, yt[1].float().view(xt[1].shape[0], -1))
        loss2.backward()
        for j, (p1, p2)  in enumerate(zip(list(LR_net.parameters()), list(copied_model.parameters()))):
            d_p1 = p1.grad.data
            d_p2 = p2.grad.data
            if weight_decay != 0:
                d_p1.add_(p1.data, alpha = weight_decay)
                d_p2.add_(p2.data, alpha = weight_decay)
            indices = torch.zeros_like(torch.clone(d_p1).detach())
            indices = indices.bernoulli_(p = prob[0]).to(torch.bool)
            d_p1.masked_fill_(~indices, 0)
            d_p2.masked_fill_(indices, 0)
            d_p1.mul_(1/b).mul_(1/prob[0])
            d_p2.mul_(1/b).mul_(1/prob[1])
            ggd_1 = torch.clone(d_p1).detach() + torch.clone(d_p2).detach()
            exp_avg = h_0[j]
            exp_avg_sq = v_0[j]
            exp_avg.mul_(beta_1).add_(ggd_1, alpha = 1 - beta_1)
            exp_avg_sq.mul_(beta_2).addcmul_(ggd_1, ggd_1.conj(), value = 1 - beta_2)
            bias_correction1 = 1 - torch.pow(beta_1, (batch_idx+1))
            bias_correction2 = 1 - torch.pow(beta_2, (batch_idx+1))
            step_size = lr / bias_correction1
            step_size_neg = step_size.neg()
            bias_correction2_sqrt = bias_correction2.sqrt()
            denom = (exp_avg_sq.sqrt() / (bias_correction2_sqrt * step_size_neg)).add_(sigma / step_size_neg)
            p1.data.addcdiv_(exp_avg, denom)
        copied_model = copy.deepcopy(LR_net)
        copied_model.to(device)
        batch_idx += 1
    if (epoch+1) % max_batch_size == 0:
        LR_net.eval()
        current_gradnorm = total_grad(LR_net, loss_func_rec, ijcnn_train_loader, n)
        ijcnn_gadam_gradnorm_list.append(current_gradnorm)
        with torch.no_grad():
            current_loss = total_loss(LR_net, loss_func_rec, ijcnn_train_loader, n)
            ijcnn_gadam_loss_list.append(current_loss)
            current_test_loss = test_loss(LR_net, loss_func_rec, ijcnn_test_loader, n1)
            ijcnn_gadam_test_loss_list.append(current_test_loss)
            current_iteration =  (epoch+1)/max_batch_size
        print('Iteration: {}  Loss: {}  Gradnorm:{}'.format(current_iteration, current_loss, current_gradnorm))
            

In [ ]:
#training stage: logistic regression with l^2 regularization using ggd-svrg
#stage one: preparation, initialization and hyperparameter setting
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
LR_net = LogisticRegression(input_dim = d, output_dim = 1)
Snap_model = copy.deepcopy(LR_net)
LR_net.to(device)
Snap_model.to(device)
loss_func = nn.BCELoss()
lr0 = 0.8
lr_schedule = 'constant'
b = 2
m = 16
max_batch_size = int(n/(b*m))
q = int(0.26*n)


ijcnn_gsvrg_loss_list = []
ijcnn_gsvrg_gradnorm_list = []
ijcnn_gsvrg_test_loss_list = []


#stage two: load training set and testing set
BS = BatchSampler(RandomSampler(ijcnn_train, replacement = False, num_samples = b*m), batch_size = b*m, drop_last = False)
LR_ijcnn_train_loader = DataLoader(ijcnn_train, batch_sampler = BS)
LR_ijcnn_train_loader = DeviceDataLoader(LR_ijcnn_train_loader, device)


#stage three: train and test 
fg = full_grad(Snap_model, loss_func_rec, ijcnn_train_loader, n)
batch_idx = 0

for epoch in range(epoches*max_batch_size):
    LR_net.train()
    for y_target, x_data in LR_ijcnn_train_loader:
        g0 = []
        g1 = []
        gf_r0 = []
        gf_r1 = []
        ggd = []
        xt = []
        yt = []
        losst = torch.empty(2)
        xt = x_data.split(m, dim = 0)
        yt = y_target.split(m, dim = 0)
        if lr_schedule == 't-inverse':
            lr = lr0 * (1/int(1 + epoch + batch_idx/n))
        else:
            lr = lr0
        #construct the svrg-based grafting gradient
        #prepare for g_mb(bar{x})
        for i, x in enumerate(xt):
            output = Snap_model(x)
            loss_snap = loss_func(output, yt[i].float().view(x.shape[0], -1))
            loss_snap.backward()
            for j, p in enumerate(list(Snap_model.parameters())):
                d_p = p.grad.data
                if weight_decay != 0:
                    d_p.add_(p.data, alpha = weight_decay)
                if i == 0:
                    gf_r0.append(torch.clone(d_p).detach())
                else:
                    gf_r1.append(torch.clone(d_p).detach())
            zero_grad(list(Snap_model.parameters()))
        norm_2 = torch.zeros(2)
        #deriving the sampling probability and preparing for g_mb(x^k_s)
        for i, x in enumerate(xt):
            output = LR_net(x)
            loss = loss_func(output, yt[i].float().view(x.shape[0], -1))
            loss.backward()
            if i == 0:
                for z, p in zip(gf_r0, list(LR_net.parameters())):
                    d_p = p.grad.data
                    if weight_decay != 0:
                        d_p.add_(p.data, alpha = weight_decay)
                    g0.append(torch.clone(d_p).detach())
                    
                    norm_2[i] = norm_2[i] + torch.sum(torch.square(torch.add(z, torch.clone(d_p).detach(), alpha = -1)))
            else:
                for z, p in zip(gf_r1, list(LR_net.parameters())):
                    d_p = p.grad.data
                    if weight_decay != 0:
                        d_p.add_(p.data, alpha = weight_decay)
                    g1.append(torch.clone(d_p).detach())
                    
                    norm_2[i] = norm_2[i] + torch.sum(torch.square(torch.add(z, torch.clone(d_p).detach(), alpha = -1)))
            zero_grad(list(LR_net.parameters()))
        if torch.min(norm_2) == 0:
            norm_2 = torch.ones(2)
        prob = torch.sqrt(norm_2)/torch.sum(torch.sqrt(norm_2))
       
        #constructing the grafting gradient \tilde{g}^k_mb
        for qr, qo, pr, po, fg_p in zip(gf_r0, g0, gf_r1, g1, fg):
            indices = torch.zeros_like(qr)
            indices = indices.bernoulli_(p = prob[0]).to(torch.bool)
            qr.masked_fill_(~indices, 0)
            qo.masked_fill_(~indices, 0)
            pr.masked_fill_(indices, 0)
            po.masked_fill_(indices, 0)
            ggd.append(torch.add(po, qo) - torch.add(pr, qr) + fg_p)
        
        #update!
        for g, p in zip(ggd, list(LR_net.parameters())):
            p.data = torch.add(p.data, g, alpha = -lr)
        batch_idx += 1
        #Break the loop when iteration number equals to update frequency
    if  batch_idx  % q == 0:
        Snap_model = copy.deepcopy(LR_net)
        fg = full_grad(Snap_model, loss_func_rec, ijcnn_train_loader, n)
            
    if batch_idx % max_batch_size == 0:
        LR_net.eval()
        current_gradnorm = total_grad(LR_net, loss_func_rec, ijcnn_train_loader, n)
        ijcnn_gsvrg_gradnorm_list.append(current_gradnorm)
        with torch.no_grad():
            current_loss = total_loss(LR_net, loss_func_rec, ijcnn_train_loader, n)
            ijcnn_gsvrg_loss_list.append(current_loss)
            current_test_loss = test_loss(LR_net, loss_func_rec, ijcnn_test_loader, n1)
            ijcnn_gsvrg_test_loss_list.append(current_test_loss)
            current_iteration = batch_idx / max_batch_size
            print('Iteration: {}  Loss: {}  Gradnorm:{}'.format(current_iteration, current_loss, current_gradnorm))

        